In [11]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np


In [12]:
# Load Pretrained Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model initially

# Add task-specific layers with improved Dropout
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3), 
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5), 
    layers.Dense(4, activation='softmax')  # Adjust number of classes accordingly
])

# Implement Learning Rate Decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

# Compile the model with improved learning rate tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)


In [13]:
AUTOTUNE = tf.data.AUTOTUNE

# Function to preprocess images
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

# Load datasets using tf.data pipeline
train_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    image_size=(224, 224),
    batch_size=32
).map(preprocess_image).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/val',
    image_size=(224, 224),
    batch_size=32
).map(preprocess_image).cache().prefetch(buffer_size=AUTOTUNE)


Found 3919 files belonging to 4 classes.
Found 395 files belonging to 4 classes.


In [14]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint_callback = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stopping, checkpoint_callback],
    verbose=1
)

# Fine-tune the entire model
base_model.trainable = True  # Unfreeze the base model

# Recompile model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',  # Or categorical if using one-hot
    metrics=['accuracy']
)

# Fine-tune the model with a new EarlyStopping instance
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stopping, checkpoint_callback],  # New callbacks
    verbose=1
)

Epoch 1/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 0.6264 - loss: 0.9542
Epoch 1: val_loss improved from inf to 0.45297, saving model to best_model.keras
123/123 ━━━━━━━━━━━━━━━━━━━━ 42s 285ms/step - accuracy: 0.6273 - loss: 0.9521 - val_accuracy: 0.8557 - val_loss: 0.4530
Epoch 2/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.8291 - loss: 0.4603
Epoch 2: val_loss did not improve from 0.45297
123/123 ━━━━━━━━━━━━━━━━━━━━ 32s 257ms/step - accuracy: 0.8292 - loss: 0.4602 - val_accuracy: 0.8380 - val_loss: 0.4658
Epoch 3/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 0.8586 - loss: 0.3722
Epoch 3: val_loss improved from 0.45297 to 0.41351, saving model to best_model.keras
123/123 ━━━━━━━━━━━━━━━━━━━━ 39s 315ms/step - accuracy: 0.8585 - loss: 0.3723 - val_accuracy: 0.8835 - val_loss: 0.4135
Epoch 4/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.8651 - loss: 0.3761
Epoch 4: val_loss did not improve from 0.41351
123/123 ━━━━━━━━━━━━━━━━━━━━